# Riza Code Interpreter

> The Riza Code Interpreter is a WASM-based isolated environment for running Python or JavaScript generated by AI agents.

In this notebook we'll create an example of an agent that uses Python to solve a problem that an LLM can't solve on its own:
counting the number of 'r's in the word "strawberry."

Before you get started grab an API key from the [Riza dashboard](https://dashboard.riza.io). For more guides and a full API reference
head over to the [Riza Code Interpreter API documentation](https://docs.riza.io).

Make sure you have the necessary dependencies installed.

In [ ]:
%pip install --upgrade --quiet langchain-community rizaio

Set up your API keys as an environment variable.

In [ ]:
%env ANTHROPIC_API_KEY=<your_anthropic_api_key_here>
%env RIZA_API_KEY=<your_riza_api_key_here>

In [13]:
from langchain_community.tools.riza.command import ExecPython

In [14]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

Initialize the `ExecPython` tool.

In [15]:
tools = [ExecPython()]

Initialize an agent using Anthropic's Claude Haiku model.

In [16]:
llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use a tool if you need to solve a problem.",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
# Ask a tough question
result = agent_executor.invoke({"input": "how many rs are in strawberry?"})
print(result["output"][0]["text"])



> Entering new AgentExecutor chain...

Invoking: `riza_exec_python` with `{'code': 'word = "strawberry"\nprint(word.count("r"))'}`
responded: [{'id': 'toolu_01JwPLAAqqCNCjVuEnK8Fgut', 'input': {}, 'name': 'riza_exec_python', 'type': 'tool_use', 'index': 0, 'partial_json': '{"code": "word = \\"strawberry\\"\\nprint(word.count(\\"r\\"))"}'}]

3
[{'text': '\n\nThe word "strawberry" contains 3 "r" characters.', 'type': 'text', 'index': 0}]

> Finished chain.


The word "strawberry" contains 3 "r" characters.
